In [1]:

import torch
import torch_directml
import torch.nn as nn
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader

In [2]:
data = pd.read_csv("Resultados-MegaSena.csv", encoding='latin-1')
data.head()

,Conc,Data,NR1,NR2,NR3,NR4,NR5,NR6
0,2601,14/06/2023,3,8,34,40,44,55
1,2600,10/06/2023,4,18,37,38,46,60
2,2599,07/06/2023,23,28,34,43,47,60
3,2598,03/06/2023,7,14,24,53,58,60
4,2597,31/05/2023,14,26,34,54,56,58


In [3]:
# Prepare the data
data['Data'] = pd.to_datetime(data['Data'], format='%d/%m/%Y')
data['Day'] = data['Data'].dt.day
data['Month'] = data['Data'].dt.month
data = data.drop('Data', axis=1)

In [4]:
# Define the dataset
class LotteryDataset(Dataset):
    def __init__(self, data):
        self.X = data[['Conc', 'NR1', 'NR2', 'NR3', 'NR4', 'NR5', 'NR6', 'Day', 'Month']].values
        self.y = data[['NR1', 'NR2', 'NR3', 'NR4', 'NR5', 'NR6']].values
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return torch.tensor(self.X[idx]).float(), torch.tensor(self.y[idx]).float()

In [5]:
# Define the model architecture
class LotteryNet(nn.Module):
    def __init__(self):
        super(LotteryNet, self).__init__()
        
        self.fc1 = nn.Linear(9, 32)  # Input layer (9 features)
        self.fc2 = nn.Linear(32, 64) # Hidden Layer 1
        self.fc3 = nn.Linear(64, 128) # Hidden Layer 2
        self.fc4 = nn.Linear(128, 64) # Hidden Layer 3
        self.fc5 = nn.Linear(64, 6) # Output layer
        
        self.relu = nn.ReLU() # Activation function
        
    def forward(self, x):
        out = self.relu(self.fc1(x))
        out = self.relu(self.fc2(out))
        out = self.relu(self.fc3(out))
        out = self.relu(self.fc4(out))
        out = self.fc5(out)
        return out

In [6]:
# Prepare the model for training
# device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device = torch.device(torch_directml.device())
model = LotteryNet().to(device)
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [7]:
# Initialize the data loader
batch_size = 16
dataset = LotteryDataset(data)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [8]:
# Train the model
num_epochs = 100
for epoch in range(num_epochs):
    for i, (inputs, targets) in enumerate(dataloader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        
        outputs = model(inputs)
        loss = loss_fn(outputs, targets)
        loss.backward()
        
        optimizer.step()
        
        if i % 20 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(dataloader)}], Loss: {loss.item():.4f}')

Epoch [1/100], Step [1/163], Loss: 1730.2454
Epoch [1/100], Step [21/163], Loss: 399.1405
Epoch [1/100], Step [41/163], Loss: 319.4892
Epoch [1/100], Step [61/163], Loss: 86.9622
Epoch [1/100], Step [81/163], Loss: 64.7542
Epoch [1/100], Step [101/163], Loss: 60.0641
Epoch [1/100], Step [121/163], Loss: 40.4751
Epoch [1/100], Step [141/163], Loss: 37.5550
Epoch [1/100], Step [161/163], Loss: 34.3356
Epoch [2/100], Step [1/163], Loss: 32.5176
Epoch [2/100], Step [21/163], Loss: 40.1238
Epoch [2/100], Step [41/163], Loss: 47.6702
Epoch [2/100], Step [61/163], Loss: 36.0490
Epoch [2/100], Step [81/163], Loss: 43.8821
Epoch [2/100], Step [101/163], Loss: 34.1641
Epoch [2/100], Step [121/163], Loss: 41.3649
Epoch [2/100], Step [141/163], Loss: 33.1075
Epoch [2/100], Step [161/163], Loss: 41.1636
Epoch [3/100], Step [1/163], Loss: 40.9848
Epoch [3/100], Step [21/163], Loss: 52.1539
Epoch [3/100], Step [41/163], Loss: 36.5810
Epoch [3/100], Step [61/163], Loss: 32.7996
Epoch [3/100], Step [81

In [9]:
# Define the contest number and extract the last 6 winning numbers
contest_number = data['Conc'].max() + 1 # assuming the next contest number is one greater than the max contest number in the dataset
last_six_numbers = data.loc[data['Conc'] == contest_number - 1, ['NR1', 'NR2', 'NR3', 'NR4', 'NR5', 'NR6']].values.ravel()

# Generate next 6 numbers
next_six_numbers = []
while len(next_six_numbers) < 6:
    num = np.random.randint(1, 61) # assuming Mega Sena has 60 balls
    if num not in last_six_numbers and num not in next_six_numbers:
        next_six_numbers.append(num)

print(f"The next 6 numbers for contest {contest_number} of Mega Sena are: {next_six_numbers}")

The next 6 numbers for contest 2602 of Mega Sena are: [7, 24, 14, 6, 50, 5]
